In [1]:
from bson import ObjectId
import pandas as pd
import numpy as np
# File paths
movies_file_path = 'movies.csv'
links_file_path = 'links.csv'
ratings_file_path = 'ratings.csv'
tags_file_path = 'tags.csv'

# Loading each file into a DataFrame
movies_df = pd.read_csv(movies_file_path)
links_df = pd.read_csv(links_file_path)
ratings_df = pd.read_csv(ratings_file_path)
tags_df = pd.read_csv(tags_file_path)


In [2]:
# merge the ratings and movies dataframes 
merged_df = pd.merge(ratings_df, movies_df, on='movieId')

# convert genre in array
merged_df['genres'] = merged_df['genres'].apply(lambda x: x.split('|'))

In [3]:
import datetime

# Convert the timestamp from UNIX time to a human-readable date format
merged_df['timestamp'] = pd.to_datetime(merged_df['timestamp'], unit='s')

# Display the first few rows of the modified dataframe to confirm the transformation
merged_df.tail()


,userId,movieId,rating,timestamp,title,genres
100831,610,160341,2.5,2016-11-19 08:55:49,Bloodmoon (1997),"[Action, Thriller]"
100832,610,160527,4.5,2016-11-19 08:43:18,Sympathy for the Underdog (1971),"[Action, Crime, Drama]"
100833,610,160836,3.0,2017-05-03 20:53:14,Hazard (2005),"[Action, Drama, Thriller]"
100834,610,163937,3.5,2017-05-03 21:59:49,Blair Witch (2016),"[Horror, Thriller]"
100835,610,163981,3.5,2017-05-03 22:22:35,31 (2016),[Horror]


In [4]:
# Normalizing the ratings on a scale from 0 to 1
merged_df['rating'] = merged_df['rating'] / 5.0

# Display the first few rows of the dataframe to confirm the normalization
merged_df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,0.8,2000-07-30 18:45:03,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,5,1,0.8,1996-11-08 06:36:02,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
2,7,1,0.9,2005-01-25 06:52:26,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
3,15,1,0.5,2017-11-13 12:59:30,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
4,17,1,0.9,2011-05-18 05:28:03,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"


In [5]:
# Convert the timestamp from UNIX time to a human-readable date format
tags_df['timestamp'] = pd.to_datetime(tags_df['timestamp'], unit='s')

# Display the first few rows of the modified dataframe to confirm the transformation
tags_df.head()

,userId,movieId,tag,timestamp
0,2,60756,funny,2015-10-24 19:29:54
1,2,60756,Highly quotable,2015-10-24 19:29:56
2,2,60756,will ferrell,2015-10-24 19:29:52
3,2,89774,Boxing story,2015-10-24 19:33:27
4,2,89774,MMA,2015-10-24 19:33:20


In [6]:
merged_df['release'] = merged_df['title'].str.extract(r'\((\d{4})\)', expand=False)
merged_df['title'] = merged_df['title'].str.replace(r' \(\d{4}\)', '', regex=True)

merged_df['release'] = pd.to_numeric(merged_df['release'], errors='coerce')


In [7]:

merged_csv_path = 'merged_movies_ratings.json'
#merged_df.to_csv(merged_csv_path, index=False)
#merged_df.to_json(merged_csv_path, orient='records', lines=True)
import json


merged_df.to_csv('merged_movies_ratings.csv', index=False, date_format='%Y-%m-%d %H:%M:%S')


merged_df_json_compatible = merged_df.where(pd.notnull(merged_df), None)

json_data = merged_df_json_compatible.to_dict(orient='records')

with open('merged_movies_ratings.json', 'w') as json_file:
    json.dump(json_data, json_file, default=str)

merged_csv_path
merged_df.tail()

,userId,movieId,rating,timestamp,title,genres,release
100831,610,160341,0.5,2016-11-19 08:55:49,Bloodmoon,"[Action, Thriller]",1997.0
100832,610,160527,0.9,2016-11-19 08:43:18,Sympathy for the Underdog,"[Action, Crime, Drama]",1971.0
100833,610,160836,0.6,2017-05-03 20:53:14,Hazard,"[Action, Drama, Thriller]",2005.0
100834,610,163937,0.7,2017-05-03 21:59:49,Blair Witch,"[Horror, Thriller]",2016.0
100835,610,163981,0.7,2017-05-03 22:22:35,31,[Horror],2016.0
